In [4]:
import torch
import numpy as np
from scipy.io import savemat
from torch_geometric.utils import to_dense_adj

In [5]:
dataset = torch.load('data/dataset_python_node.pt', weights_only=False) 

In [8]:
# # Load dataset
# dataset = torch.load('data/dataset_python_node.pt', weights_only=False) 

# # Parameter: keep only the first n_keep features from each node's feature vector.
# # Adjust this value to drop the remaining features.
# n_keep = 1  # for example, keep only the first 50 features

# # Prepare plain Python lists for each attribute
# edge_indices = []
# features = []
# labels = []

# # Iterate through each Data object in the dataset
# for data in dataset:
#     # Convert to NumPy arrays if needed
#     edge_index = data.edge_index.numpy() if isinstance(data.edge_index, torch.Tensor) else data.edge_index
#     feature = data.x.numpy() if isinstance(data.x, torch.Tensor) else data.x
#     # Optionally drop some features, here keeping only the first n_keep columns
#     if feature.shape[1] > n_keep:
#         feature = feature[:, :n_keep]
        
#     edge_index_tensor = torch.tensor(edge_index, dtype=torch.long)
#     adj_matrix = to_dense_adj(edge_index_tensor)[0].numpy()
#     edge_index = adj_matrix
    
#     label = data.y.numpy() if isinstance(data.y, torch.Tensor) else data.y
#     edge_indices.append(edge_index)
#     features.append(feature)
#     labels.append(label)

# # Manually create object arrays to avoid broadcasting issues
# edge_indices_obj = np.empty(len(edge_indices), dtype=object)
# features_obj = np.empty(len(features), dtype=object)
# labels_obj = np.empty(len(labels), dtype=object)

# for i in range(len(edge_indices)):
#     edge_indices_obj[i] = edge_indices[i]
#     features_obj[i] = features[i]
#     labels_obj[i] = labels[i]

# # Build the dictionary for saving to .mat
# data_dict = {
#     'edge_indices': edge_indices_obj,  # MATLAB cell array equivalent
#     'features': features_obj,          # Object array of (possibly reduced) feature matrices
#     'labels': labels_obj               # Object array of labels
# }

# # Save to .mat file
# savemat('data/node.mat', data_dict)
# print("Conversion to .mat complete!")


Conversion to .mat complete!


In [11]:
# Prepare plain Python lists for each attribute
edge_indices = []
features = []
labels = []

# Iterate through each Data object in the dataset
for data in dataset:
    # Convert to NumPy arrays if needed
    edge_index = data.edge_index.numpy() if isinstance(data.edge_index, torch.Tensor) else data.edge_index
    feature = data.x.numpy() if isinstance(data.x, torch.Tensor) else data.x
    edge_index_tensor = torch.tensor(edge_index, dtype=torch.long)
    adj_matrix = to_dense_adj(edge_index_tensor)[0].numpy()
    edge_index = adj_matrix
    label = data.y.numpy() if isinstance(data.y, torch.Tensor) else data.y

    num_nodes = feature.shape[0]

    edge_indices.append(edge_index)
    features.append(feature)
    labels.append(label)

# Manually create object arrays to avoid broadcasting issues
edge_indices_obj = np.empty(len(edge_indices), dtype=object)
features_obj = np.empty(len(features), dtype=object)
labels_obj = np.empty(len(labels), dtype=object)

for i in range(len(edge_indices)):
    edge_indices_obj[i] = edge_indices[i]
    features_obj[i] = features[i]
    labels_obj[i] = labels[i]

# Build the dictionary for saving to .mat
data_dict = {
    'edge_indices': edge_indices_obj,  # Now an object array (MATLAB cell array)
    'features': features_obj,          # Object array of feature matrices
    'labels': labels_obj               # Object array of labels
}

# Save to .mat file
savemat('data/node.mat', data_dict)
print("Conversion to .mat complete!")

Conversion to .mat complete!
